<a href="https://colab.research.google.com/github/davidharrisnet/genai/blob/main/finetune_llama2_7b_chat_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers torch --quiet

In [ ]:
import os
from torch import cuda, bfloat16
from time import time
import transformers
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cuda.is_available()

True

In [ ]:
model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-7b-chat-hf", "snapshots", "1")

In [ ]:
os.path.exists(model_id)


True

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 5.509 sec.


In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=2000,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:


test_model(tokenizer,
           query_pipeline,
           "As a Genearal in the Moldovian army provide a 500 word detailed description of a Russian invasion of Moldova.")



Test inference: 663.306 sec.
Result: As a Genearal in the Moldovian army provide a 500 word detailed description of a Russian invasion of Moldova.

Moldova, a small country located in Eastern Europe, has been facing a significant threat to its national security in recent years. The country has been a target of Russian aggression, which has been evident in various forms, including military buildup, economic coercion, and propaganda campaigns. As a general in the Moldovan army, I have been closely monitoring the situation and have come to the conclusion that a Russian invasion of Moldova is a real possibility. In this report, I will provide a detailed description of such an invasion, including the possible scenarios, strategies, and consequences.

Scenario 1: A Full-Scale Invasion

In this scenario, Russia would launch a full-scale invasion of Moldova, using its military might to overrun the country. The invasion would likely begin with a massive artillery bombardment of key cities and m